In [1]:
import pandas as pd
import mysql.connector
import numpy as np

In [2]:

db_config={
        'host' : 'Localhost',
        'user' : 'root',
        'password' :'munene2024'
        }

In [3]:
conn=mysql.connector.connect(**db_config)

In [4]:
def execute_query(query):
    cursor=conn.cursor()
    cursor.execute(query)
    conn.commit()
    cursor.close()

In [5]:
create_db_query='CREATE DATABASE IF NOT EXISTS ecommerce_db;'
execute_query(create_db_query)

In [6]:
use_db_query = "USE ecommerce_db;"
execute_query(use_db_query)

In [14]:
    """Creates the 'products' table if it doesn't exist."""
    try:
        cursor = conn.cursor()
        query = """
        CREATE TABLE IF NOT EXISTS products (
            product_id VARCHAR(255),
            product_name VARCHAR(255),
            category TEXT,
            discounted_price DECIMAL(10, 2),
            actual_price DECIMAL(10, 2),
            discount_percentage DECIMAL(5, 2),
            rating DECIMAL(3, 2),
            rating_count INT,
            about_product TEXT,
            user_id VARCHAR(255),
            user_name VARCHAR(255),
            review_id VARCHAR(255),
            review_title TEXT,
            review_content TEXT,
            img_link TEXT,
            product_link TEXT
        );
        """
        cursor.execute(query)
        conn.commit()
        print("Products table created or already exists.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if conn and conn.is_connected():
            cursor.close()

Products table created or already exists.


In [8]:
import pandas as pd
from sqlalchemy import create_engine

def load_csv_to_db(csv_file):
    db_config = {'host': 'localhost', 'user': 'root', 'password': 'munene2024', 'database': 'ecommerce_db'}
    try:
        engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")
        pd.read_csv(csv_file).to_sql('products', con=engine, if_exists='replace', index=False)
        print(f"Loaded {csv_file}")
    except Exception as e:
        print(f"Error: {e}")

load_csv_to_db("updated_data.csv")

Loaded updated_data.csv


In [9]:
try:
    engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/ecommerce_db")
    df = pd.read_sql_query("SELECT product_name, rating_count FROM products ORDER BY rating_count DESC LIMIT 10", engine)
    if not df.empty:
        for _, row in df.iterrows():
            print(f"Product: {row['product_name']}, Reviews: {row['rating_count']}")
    else:
        print("No products found.")
except Exception as e:
    print(f"Error: {e}")

Product: Eureka Forbes car Vac 100 Watts Powerful Suction Vacuum Cleaner with Washable HEPA Filter, 3 Accessories,Compact,Light Weight & Easy to use (Black and Red), Reviews: 992
Product: Eveready Red 1012 AAA Batteries - Pack of 10, Reviews: 989
Product: Sennheiser CX 80S in-Ear Wired Headphones with in-line One-Button Smart Remote with Microphone Black, Reviews: 98,250
Product: Pigeon 1.5 litre Hot Kettle and Stainless Steel Water Bottle Combo used for boiling Water, Making Tea and Coffee, Instant Noodles, Soup, 1500 Watt with Auto Shut- off Feature - (Silver), Reviews: 976
Product: Zoul USB C 60W Fast Charging 3A 6ft/2M Long Type C Nylon Braided Data Cable Quick Charger Cable QC 3.0 for Samsung Galaxy M31S M30 S10 S9 S20 Plus, Note 10 9 8, A20e A40 A50 A70 (2M, Grey), Reviews: 974
Product: Zoul USB C 60W Fast Charging 3A 6ft/2M Long Type C Nylon Braided Data Cable Quick Charger Cable QC 3.0 for Samsung Galaxy M31S M30 S10 S9 S20 Plus, Note 10 9 8, A20e A40 A50 A70 (2M, Grey), Review

In [10]:
try:
    query = """
    SELECT category, AVG(rating) AS average_rating
    FROM products
    GROUP BY category
    ORDER BY average_rating DESC
    LIMIT 1;
    """
    df = pd.read_sql_query(query, engine)
    if not df.empty:
        category = df.iloc[0]['category']
        average_rating = df.iloc[0]['average_rating']
        print(f"Category with highest average rating: {category}, Average Rating: {average_rating:.2f}")
    else:
        print("No products found.")
except Exception as e:
    print(f"Error: {e}")

Category with highest average rating: Computers&Accessories|Tablets, Average Rating: 4.60


In [11]:
try:
    query = """
    SELECT product_name, discount_percentage
    FROM products
    WHERE discount_percentage > 50;
    """
    df = pd.read_sql_query(query, engine)
    if not df.empty:
        print("Products with discounts greater than 50%:")
        for index, row in df.iterrows():
            print(f"Product: {row['product_name']}, Discount: {row['discount_percentage']}%")
    else:
        print("No products with discounts greater than 50% found.")
except Exception as e:
    print(f"Error: {e}")

Products with discounts greater than 50%:
Product: Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey), Discount: 64%%
Product: Sounce Fast Phone Charging Cable & Data Sync USB Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini & iOS Devices, Discount: 90%%
Product: boAt Deuce USB 300 2 in 1 Type-C & Micro USB Stress Resistant, Tangle-Free, Sturdy Cable with 3A Fast Charging & 480mbps Data Transmission, 10000+ Bends Lifespan and Extended 1.5m Length(Martian Red), Discount: 53%%
Product: Portronics Konnect L 1.2M Fast Charging 3A 8 Pin USB Cable with Charge & Sync Function for iPhone, iPad (Grey), Discount: 61%%
Product: pTron Solero TB301 3A Type-C Data and Fast Charging Cable, Made in India, 480Mbps Data Sync, Strong and Durable 1.5-Meter Nylon Braided USB Cable for Type-C Devices for Charging Adapter (Black), Discount: 85%%
Product: boAt Micro USB 55 T

In [12]:
try:
    query = """
    SELECT user_name, COUNT(*) AS review_count
    FROM products
    GROUP BY user_name
    ORDER BY review_count DESC
    LIMIT 1;
    """
    df = pd.read_sql_query(query, engine)
    if not df.empty:
        user_name = df.iloc[0]['user_name']
        review_count = df.iloc[0]['review_count']
        print(f"User with most reviews: {user_name}, Review Count: {review_count}")
    else:
        print("No reviews found.")
except Exception as e:
    print(f"Error: {e}")

User with most reviews: $@|\|TO$|-|,Sethu madhav,Akash Thakur,Burger Planet,Justice ⚖️,indrajyoti d.,Aditya Kumar,E.C.GEORGE, Review Count: 10


In [13]:
try:
    query = """
    SELECT category, SUM(rating_count) AS total_reviews
    FROM products
    GROUP BY category
    ORDER BY total_reviews DESC
    LIMIT 5;
    """
    df = pd.read_sql_query(query, engine)
    if not df.empty:
        print("Top 5 Best-Selling Categories (based on number of reviews):")
        for index, row in df.iterrows():
            print(f"Category: {row['category']}, Total Reviews: {row['total_reviews']}")
    else:
        print("No categories found.")
except Exception as e:
    print(f"Error: {e}")

Top 5 Best-Selling Categories (based on number of reviews):
Category: Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables, Total Reviews: 27545.0
Category: Electronics|HomeTheater,TV&Video|Accessories|RemoteControls, Total Reviews: 12692.0
Category: Electronics|WearableTechnology|SmartWatches, Total Reviews: 6297.0
Category: Home&Kitchen|Heating,Cooling&AirQuality|WaterHeaters&Geysers|InstantWaterHeaters, Total Reviews: 5339.0
Category: Electronics|HomeTheater,TV&Video|Televisions|SmartTelevisions, Total Reviews: 3942.0
